In [1]:
import course;course.header()

# Advanced Python Course 
## Mobi Heidelberg WS 2021/22
### by Christian Fufezan 

christian@fufezan.net

https://fufezan.net

<img src="./images/cc.png" alt="drawing" width="200" style="float: left;"/>


In [2]:
course.display_topics(day=5)

,day,Topic
19,5,Test Driven development
20,5,Auto documentation with Sphinx


# Test Driven Development


## You all are already do some part of test driven development!

<img style="right" src="./images/tdd_already.jpg">

## TDD 
Test driven development comes in three blocks.

<img style="right" src="./images/tdd_explained.png" width=700>

a) you start thinking about what you want to code, write a test that will fail and then start writing the code to make the test pass. At this stage you have a working prototype and leave the first block (blue dotted line)

b) you will write more tests covering foreseeable corner cases. If the test fail you adjust your code. After the tests pass you will leave the second block behind having a real product with test. 

c) Finally you start refactoring your code into the bigger picture. The tests will give you confidence that nothing breaks while you refactor. This step becomes essential when you tackle a similar problem or consolidate functionality in your core package. After that, you leave the third block and your code is maintainable, reusable and generalized in a bigger context. 

Advantages are
* think before coding. In a later stage you will think where the code will sit, how it wil work with the rest of your code base.
* you will write tests before actually start coding on the solution (ie you will have tests at the end of the dev phase)
* Forces you to write smaller functions that so TDD enforces the Zen of Python and general* coding philosophy
    * Explicit is better than implicit.
    * Simple is better than complex.
    * Complex is better than complicated.
    * a function should do one thing and one thing only



In order to do so, we need to 
* fork the advanced_python_repo (so you can commit you code to your repo)
* prepare some folders in your repo using this ...
* if you can, switch to terminal and activate our virtualenv

### Create folder structure

In [3]:
!mkdir ../peak_finder
!mkdir ../tests
!mkdir ../docs

You should have a dir structure like this:

    ├── LICENSE
    ├── README.md
    ├── data
    │   ├── ...
    ├── docs
    ├── notebooks
    │   ├── ...
    ├── peak_finder
    └── tests


### Install pytest!

In [3]:
!pip install --upgrade pip
!pip install pytest
# you need to restart VSCode :/

  Using cached pip-21.3.1-py3-none-any.whl (1.7 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.1.1
    Uninstalling pip-20.1.1:
      Successfully uninstalled pip-20.1.1


### Create basic peak_finder place holders 

In [2]:
%%writefile ../peak_finder/__init__.py
# __init__ py is required in a folder 
# to be recognized as a python module
# otherwise the import statements won't work
# the %%writefile magic allows the jupyter cell content to be stored as a file

# lets load core into the name space as well
from . import basic

Overwriting ../peak_finder/__init__.py


In [5]:
%%writefile ../peak_finder/basic.py
#
# The first version of our function!
# Write doc strings 
#
def find_peaks(list_of_intensities):
    """Find peaks

    Find local maxima for a given list of intensities or tuples
    Intensities are defined as local maxima if the
    intensities of the elements in the list before and after
    are smaller than the peak we want to determine.

    For example given a list:
        1 5 [6] 4 1 2 [3] 2

    We expect 6 and 3 to be returned.
    

    Args:
        list_of_intensities (list of floats, ints or tuple of ints): a list of
            numeric values

    Returns:
        list of floats or tuples: list of the identified local maxima

    Note:
        This is just a place holder for the TDD part :)

    """
    return


Writing ../peak_finder/basic.py


### Write first test!

In [3]:
%%writefile ../tests/test_basic.py
import sys
from pathlib import Path
# -------- START of inconvenient addon block --------
# This block is not necessary if you have installed your package
# using e.g. pip install -e (requires setup.py)
# or have a symbolic link in your sitepackages (my preferend way)
sys.path.append(
    str(Path(__file__).parent.parent.resolve())
)
# It make import peak_finder possible
# This is a demo hack for the course :)
# --------  END of inconvenient addon block  --------

import peak_finder

def test_find_peaks():
    peaks = peak_finder.basic.find_peaks([0, 2, 1])
    assert peaks == [2] 

Overwriting ../tests/test_basic.py


<image src="./images/VSCode/tdd_first_fail.png">

## Fix it!

## Now let's go into the first (second) iterations

### on the path to our first product

<img style="right" src="images/tdd_explained.png" width=800>

## Now let's make it brilliant


Let's say we are happy with our *product* and got "rich". 

Now why do we need to refactor?

* Not to have our code exist in "peak_finder" but move it into our *work horse* package
* The moment the definition of list_of_intensities is altered, we would 
    * restart the TDD process at the start  
    * remember that both function do something similar so we could ultimately merge their code

## Lets go into the third iteration


### case find_peaks in a vector filled with colors

Colors are defined as (e.g.) red-green-blue (RGB) tuples. So (0, 0, 0) is black and (255, 255, 255) is white.

<img src="https://www.alanzucconi.com/wp-content/uploads/2015/09/colours.png">

And let's not go too deep into the beautiful world of [sorting colors by Alan Zucconi](https://www.alanzucconi.com/2015/09/30/colour-sorting/) and let's just say
(20,0,0) > (0,19,0) so we sum-up the values in the tuples and feed it into our function, but this time we look for dark spots, that we want to identify as "peaks".

### Write a second function!

### What could refactoring look like ?

# Automation is key!

We want out test to run everytime a pull request is opened on github. 

Github actions enable automatic tasks upon different actions to be triggered. Read more about it [here](https://github.com/features/actions)

github actions are defined as yaml files and are placed under .github/workflows


In [6]:
!mkdir -p ../.github/workflows

In [7]:
%%writefile ../.github/workflows/pytest.yml
name: pytest with codecov
on:
  pull_request:
    types: [opened, synchronize, reopened, edited]

jobs:
  build:
    name: Run Python Tests
    runs-on: ubuntu-latest

    steps:

    - uses: actions/checkout@v2
      with:
        fetch-depth: 2

    - name: Set up Python 3.9
      uses: actions/setup-python@v2
      with:
        python-version: 3.9

    - name: Install Python dependencies
      run: |
        sudo apt install -y $(grep -o ^[^#][[:alnum:]-]* "packages.list")
        python3 -m pip install --upgrade pip
        pip3 install -r requirements.txt

    - name: Test with pytest
      run: |
        pytest --exitfirst --verbose --failed-first \
        --cov=. --cov-report xml
    
    - name: Upload Coverage to Codecov
      uses: codecov/codecov-action@v2
      with:
        token: ${{ secrets.CODECOV_TOKEN }} # not required for public repos

Writing ../.github/workflows/pytest.yml


If you have a private repo, you need to add a settings > secrets name CODECOV_TOKEN, which you can get from codecov.io - register with your github account.

After you pushed to you repo, you should have new action! 

# Documentation to our peak_finder is important

## Auto documentation using Sphinx!

"Sphinx is a tool that makes it easy to create intelligent and beautiful documentation, written by Georg Brandl and licensed under the BSD license.

It was originally created for the Python documentation, and it has excellent facilities for the documentation of software projects in a range of languages. Of course, this site is also created from reStructuredText sources using Sphinx! The following features should be highlighted:"

[Website](http://www.sphinx-doc.org/en/master/)

In [11]:
!pip install sphinx

Quickstart, open terminal and
``` bash
$ cd docs
```

**NOTE:** Personally, I do not like the docs to clutter my project dir with different files but to have everything contained in the docs folder

``` bash
 fu@mPro:~/dev/_teaching/advanced_python_2021-22_HD_pre/docs
    5 main $ sphinx-quickstart                                                                                                                                                                                                                                                    .venv [13:32:11]
Welcome to the Sphinx 4.2.0 quickstart utility.

Please enter values for the following settings (just press Enter to
accept a default value, if one is given in brackets).

Selected root path: .

You have two options for placing the build directory for Sphinx output.
Either, you use a directory "_build" within the root path, or you separate
"source" and "build" directories within the root path.
> Separate source and build directories (y/n) [n]: y

The project name will occur in several places in the built documentation.
> Project name: peak_finder
> Author name(s): Christian Fufezan
> Project release []: alpha

If the documents are to be written in a language other than English,
you can select a language here by its language code. Sphinx will then
translate text that it generates into that language.

For a list of supported codes, see
https://www.sphinx-doc.org/en/master/usage/configuration.html#confval-language.
> Project language [en]: 

Creating file /Users/fu/dev/_teaching/advanced_python_2021-22_HD_pre/docs/source/conf.py.
Creating file /Users/fu/dev/_teaching/advanced_python_2021-22_HD_pre/docs/source/index.rst.
Creating file /Users/fu/dev/_teaching/advanced_python_2021-22_HD_pre/docs/Makefile.
Creating file /Users/fu/dev/_teaching/advanced_python_2021-22_HD_pre/docs/make.bat.

Finished: An initial directory structure has been created.

You should now populate your master file /Users/fu/dev/_teaching/advanced_python_2021-22_HD_pre/docs/source/index.rst and create other documentation
source files. Use the Makefile to build the docs, like so:
   make builder
where "builder" is one of the supported builders, e.g. html, latex or linkcheck.
```

Now let's make the first documentation
```bash
make html
open build/html/index.html
```

Sphinx does not know anything about our project yet so we have to edit
**docs/source/conf.py** you will find 


``` python
# If extensions (or modules to document with autodoc) are in another directory,
# add these directories to sys.path here. If the directory is relative to the
# documentation root, use os.path.abspath to make it absolute, like shown here.
#
# import os
# import sys
# sys.path.insert(0, os.path.abspath('.'))
``` 
that block in the beginning of the conf file, uncomment it and edit the sys.path so the module can be found
``` python
dir_path = os.path.join(
    os.path.dirname(__file__),
    os.pardir,
    os.pardir,
)
sys.path.insert(0, os.path.abspath(dir_path))
```

Additionally, let's make sphinx understand google and numpy docstring! Edit
**docs/source/conf.py** again and add the napoleon extention to the extentions 
``` python
extensions = [
    'sphinx.ext.napoleon',
]
```

Wonder what so special about google's docstring ?

*Regular*
``` 
:param path: The path of the file to wrap
:type path: str
:param field_storage: The :class:`FileStorage` instance to wrap
:type field_storage: FileStorage
:param temporary: Whether or not to delete the file when the File
   instance is destructed
:type temporary: bool
:returns: A buffered writable file descriptor
:rtype: BufferedFileStorage
```

*Google python style*
```
Args:
    path (str): The path of the file to wrap
    field_storage (FileStorage): The :class:`FileStorage` instance to wrap
    temporary (bool): Whether or not to delete the file when the File
       instance is destructed

Returns:
    BufferedFileStorage: A buffered writable file descriptor
```

For more details, see [here](https://www.sphinx-doc.org/en/1.5/ext/example_google.html)

So now let's build again!
```bash
make html
```



In [ ]:
%pwd

In [ ]:
%cd '/Users/fu/dev/teaching/advanced_python_2020-21_HD_pre/docs'

In [ ]:
!make html

### Nothing to see because we have not added our module yet!

Let's edit docs/source/index.rst

```
.. playground documentation master file, created by
   sphinx-quickstart on Sun Oct 13 15:39:43 2019.
   You can adapt this file completely to your liking, but it should at least
   contain the root `toctree` directive.

Welcome to playground's documentation!
=========================================

.. toctree::
   :maxdepth: 2
   :caption: Contents:

   core


Indices and tables
==================

* :ref:`genindex`
* :ref:`modindex`
* :ref:`search`
```

**I added core to the toctree!**



and **docs/source/basic.rst** looks like:
```
.. _basic.rst:

Basic module
============

.. automodule:: peak_finder.basic
    :members:
    :undoc-members:
```

This is sufficient to have all functions in this module to be parsed and included in this documentation.

The *:undoc-members:* helps to find all functions, even the ones that have no documentation.




Now rerun the documentation building procedure
``` bash
make html
```

In [ ]:
!make html;open build/html/index.html

# Summary

* Use test driven development! Because you do it already but you do not record it! 
* Write good doc strings! Use Google style for clarity!
* Create a Sphinx documentation because it will help you not only to auto generate latest documentation but also create pdfs and have it hosted on readthedocs.org
* Automation is key


# Exercise

* Write tests to you Protein class 
* Add actions to your github account that tests the Protein class 
* Try to auto generate sphinx documentation using github actions 